# **SVM**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window
- Entropy is the measure of information contained in a state


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

In [77]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [78]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [79]:
train_data

{'melspectrogram': array([[[-2.10519694e-02, -2.64618150e-03, -1.09052584e-02, ...,
          -1.14497980e-02,  0.00000000e+00, -1.33858034e-02],
         [-1.99002311e-01, -2.20890388e-01, -2.76137829e-01, ...,
          -2.59782881e-01, -2.09464088e-01, -1.87755451e-01],
         [-2.39491284e-01, -2.51040578e-01, -2.89648235e-01, ...,
          -2.82749534e-01, -2.52160460e-01, -2.39700243e-01],
         ...,
         [-9.98796940e-01, -9.98772740e-01, -9.98712122e-01, ...,
          -9.96163070e-01, -9.97696698e-01, -9.98156428e-01],
         [-9.99429047e-01, -9.99434769e-01, -9.99466956e-01, ...,
          -9.99509513e-01, -9.99450743e-01, -9.99419093e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.80637967e-02, -3.36689858e-08, -7.33598135e-03, ...,
          -1.05369762e-02, -8.22041358e-04, -1.49706351e-02],
         [-1.93169340e-01, -2.11562306e-01, -2.58488506e-01, ...,
   

In [80]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [81]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [82]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [83]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [84]:
val_y, val_features = shuffle_data(val_labels, v_features)

## **SVM Model**

In [85]:
train_results = {}
val_results = {}

### **With all the features**

In [86]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
# A random forest model expect a 2D input of (n_samples, n_features). Since the input is 3D here, it will cause an error when passed through a RF,
# I need to flatten the training features from 3D to 2D... eg. (7105, 93 * 259)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)

training_features.shape

(12565, 2784)

In [87]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 2784)

Fit the model with training data

In [88]:
# model = SVC(kernel='rbf', C=4)
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(metric='manhattan')

param_grid = {'n_neighbors': [1, 5, 9, 15, 20,30,40]}

clf = GridSearchCV(
        model,
        param_grid,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=2,
    )

clf.fit(training_features, train_y)
display(clf.best_score_)
display(clf.best_params_)

model = model = KNeighborsClassifier(metric='manhattan', n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features, train_y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


0.8128133704735376

{'n_neighbors': 1}

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

In [89]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6051838456901748


In [66]:
train_yhat_result = model.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2060
           1       1.00      1.00      1.00      2477
           2       1.00      1.00      1.00      2203
           3       1.00      1.00      1.00       437
           4       1.00      1.00      1.00       218
           5       1.00      1.00      1.00       387
           6       1.00      1.00      1.00       167
           7       1.00      1.00      1.00       263
           8       1.00      1.00      1.00       196
           9       1.00      1.00      1.00       443
          10       1.00      1.00      1.00       191
          11       1.00      1.00      1.00       211
          12       1.00      1.00      1.00       378
          13       1.00      1.00      1.00       232
          14       1.00      1.00      1.00       188
          15       1.00      1.00      1.00      1133
          16       1.00      1.00      1.00     

In [67]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.58      0.68      0.62       455
           1       0.33      0.61      0.43       492
           2       0.64      0.45      0.53       889
           3       0.85      0.98      0.91       150
           4       0.00      0.00      0.00        67
           5       0.78      0.58      0.67        43
           6       0.18      0.17      0.17        24
           7       0.96      1.00      0.98        44
           8       0.40      0.62      0.49        50
           9       0.93      0.85      0.89       169
          10       0.50      0.34      0.40        53
          11       0.54      0.30      0.39        66
          12       0.89      0.41      0.56        59
          13       0.56      0.58      0.57        57
          14       0.51      0.66      0.57        38
          15       0.63      0.44      0.52       311
          16       0.85      0.95      0.90   

In [68]:
from sklearn.preprocessing import MinMaxScaler

In [69]:
# model = SVC(kernel='rbf', C=4)
scaler = MinMaxScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)


model = model = KNeighborsClassifier(metric='manhattan', n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

In [70]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['MinMaxScaler'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['MinMaxScaler'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.546112115732369


In [71]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.57      0.66      0.61       455
           1       0.31      0.59      0.41       492
           2       0.63      0.42      0.50       889
           3       0.85      0.98      0.91       150
           4       0.00      0.00      0.00        67
           5       0.81      0.67      0.73        43
           6       0.17      0.17      0.17        24
           7       0.81      0.98      0.89        44
           8       0.40      0.58      0.47        50
           9       0.94      0.82      0.88       169
          10       0.48      0.43      0.46        53
          11       0.43      0.27      0.33        66
          12       0.84      0.44      0.58        59
          13       0.49      0.58      0.53        57
          14       0.47      0.63      0.54        38
          15       0.57      0.38      0.46       311
          16       0.72      0.93      0.81   

In [72]:
from imblearn.over_sampling import SMOTE



In [73]:
# model = SVC(kernel='rbf', C=4)
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)

smote = SMOTE()
training_features_balanced, train_y_balanced = smote.fit_resample(training_features, train_y)

model = model = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features_balanced, train_y_balanced)

KNeighborsClassifier(n_neighbors=1, weights='distance')

In [74]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['smote'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['smote'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5958408679927667


In [75]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.66      0.71      0.68       455
           1       0.36      0.64      0.46       492
           2       0.66      0.45      0.53       889
           3       0.86      1.00      0.93       150
           4       0.00      0.00      0.00        67
           5       0.94      0.70      0.80        43
           6       0.22      0.33      0.26        24
           7       0.98      1.00      0.99        44
           8       0.42      0.64      0.50        50
           9       0.87      0.85      0.86       169
          10       0.55      0.42      0.47        53
          11       0.42      0.24      0.31        66
          12       0.91      0.49      0.64        59
          13       0.56      0.60      0.58        57
          14       0.44      0.71      0.54        38
          15       0.68      0.49      0.57       311
          16       0.91      0.91      0.91   

# Review results from all models

In [76]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
1,MinMaxScaler,1.0,0.55
0,all_features,1.0,0.57
2,smote,1.0,0.60
